In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","12g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.microsoft.sqlserver:mssql-jdbc:9.4.0.jre8") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/worker2/.ivy2/cache
The jars for the packages stored in: /home/worker2/.ivy2/jars
com.microsoft.sqlserver#mssql-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e9c739f5-fde0-4042-9757-a5076754ee2d;1.0
	confs: [default]
	found com.microsoft.sqlserver#mssql-jdbc;9.4.0.jre8 in central
:: resolution report :: resolve 374ms :: artifacts dl 6ms
	:: modules in use:
	com.microsoft.sqlserver#mssql-jdbc;9.4.0.jre8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-e9c739f5-

[ WARN] Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
CONNECTOR_TYPE='com.microsoft.sqlserver.jdbc.SQLServerDriver'
SQL_USERNAME='tyeson'
SQL_PASSWORD='Canhtoan1509'
SQL_SERVERNAME='tyeson.database.windows.net'
SQL_DBNAME='DW'
CONNECTION_STRING='jdbc:sqlserver://tyeson.database.windows.net:1433;databaseName=DW;'

In [3]:
df = spark.read.option('header','true').csv('/home/worker2/Desktop/thesis/K18/File_K18/sample_crawl_dataset/Coursera_DataScience.csv',inferSchema=True, escape = '"')

In [4]:
import pyspark.sql.functions as f
df = df.withColumn("enroll", f.regexp_replace(f.col("enroll"), ",", "").alias("enroll")) \
    .withColumn("time", f.regexp_replace(f.col("time"), " hours", "").alias("time")) \
    .withColumn("time", f.regexp_replace(f.col("time"), " hour", "").alias("time")) \
    .withColumn("fee", f.regexp_replace(f.col("fee"), "[a-zA-Z]+", "").alias("fee"))
    #.withColumn("fee", f.regexp_replace(f.col("fee"), "Enroll", "").alias("fee"))

In [5]:
from pyspark.sql.types import FloatType, LongType, IntegerType

df = df \
  .withColumn("rating" ,
              df["rating"]
              .cast(FloatType()))   \
  .withColumn("enroll",
              df["enroll"]
              .cast(LongType())) \
  .withColumn("time",df['time'].cast(IntegerType())) \
  .withColumn("fee", df['fee'].cast(IntegerType()))

In [6]:
from pyspark.sql.functions import col,when
df = df.na.fill(value=0, subset=['rating','enroll', 'fee','time'])
df = df.na.fill(value='None', subset=['link','instructor', 'levelrequirement', 'skillrequirement', 'SkillWillLearn','SkillGain', 'Subject', 'organization', 'program', 'RelationInsOrg', 'Subtitle'])

In [7]:
from pyspark.sql.functions import when,lower
df = df.withColumn('levelrequirement', lower('levelrequirement'))

In [8]:
df = df.withColumn("levelrequirement", when((df.levelrequirement != "expert") & (df.levelrequirement != "advanced") & (df.levelrequirement != 'intermediate') & (df.levelrequirement != 'beginner'),"beginner") \
                                        .when(df.levelrequirement == "advanced","expert")
                                        .otherwise(df.levelrequirement))

In [9]:
from pyspark.sql.functions import split, col, monotonically_increasing_id

df2 = df.select(col('name'),col('link'),col('rating'),col('enroll'),col('instructor'),col('time'),col('levelrequirement'),col('skillrequirement'),
                col('SkillWillLearn'),col('SkillGain'),col('Subject'),col('organization'),col('fee'),col('program'),col('RelationInsOrg'),split(col("Subtitle"),", ").alias("Subtitle"))
#df2 = df2.withColumn("id", monotonically_increasing_id())

In [10]:
df2 = df2.withColumn("instructor", f.regexp_replace(f.col("instructor"), ", PhD", "").alias("instructor")) \
    .withColumn("instructor", f.regexp_replace(f.col("instructor"), ", Ph.D.", "").alias("instructor")) \
    .withColumn("instructor", f.regexp_replace(f.col("instructor"), chr(34), "").alias("instructor"))

In [11]:
regex2 = r"^https?://.*"
df2 = df2.filter(col("link").rlike(regex2))

In [84]:
instructor = df2.select(split(col("instructor"),",").alias("instructor"))

In [85]:
from pyspark.sql.functions import explode
instructorNode = instructor.select(explode("instructor").alias("instructor"))

In [86]:
instructorNode = instructorNode.dropDuplicates(['instructor']).select(col('instructor').alias('instructorName'))

In [87]:
from pyspark.sql.functions import split, col, monotonically_increasing_id
instructorNode = instructorNode.withColumn("id", monotonically_increasing_id())

In [88]:
instructorNode = instructorNode.select(col('id').alias('instructorId'),'instructorName')

In [37]:
instructorNode.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_instructor')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [17]:
level = df2.select(col('levelrequirement').alias('levelName'))

In [18]:
level = level.dropDuplicates(['levelName']).select('levelName')

In [19]:
level = level.withColumn("levelId", monotonically_increasing_id())

In [44]:
level.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_level')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [105]:
subtitle = df2.select(explode(col("Subtitle")).alias("language"))
subtitle = subtitle.withColumn('language', lower(col('language'))) \
            .withColumn("language", f.regexp_replace(f.col("language"), "subtitles: ", "").alias("language")) \
            .withColumn("language", f.regexp_replace(f.col("language"), "subtitles:", "").alias("language")) \
            .withColumn("language", f.regexp_replace(f.col("language"), "none", "english").alias("language"))

In [106]:
subtitleNode = subtitle.dropDuplicates(['language']).select(col('language').alias('languageName'))
subtitleNode = subtitleNode.withColumn("languageId", monotonically_increasing_id())

In [23]:
subtitleNode.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_language')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [46]:
from pyspark.sql.functions import regexp_extract
regex = r"(?:https?:\/\/)?(?:[^@\n]+@)?(?:www\.)?([^:\/\n?]+)"
regex2 = r"^https?://.*"
website = df2.select(df2.link)
website = website.filter(col("link").rlike(regex2))
website = website.withColumn("website", regexp_extract(df["link"], regex, 1))
website= website.dropDuplicates(['website']).select(col('website').alias('websiteName'))
df2 = df2.withColumn("website", regexp_extract(df["link"], regex, 1))

In [47]:
website = website.withColumn("websiteId", monotonically_increasing_id())

In [31]:
website.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_website')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [49]:
df2 = df2.withColumn("timestamp", f.current_timestamp()).alias("timestamp")
course = df2.select(col('name').alias('courseName'), col('enroll') ,col('rating'), col('fee'),col('time'), col('link'), col('levelrequirement').alias('level'), col('website'),col('timestamp').alias('createDate'))

In [60]:
level_joined_df = course.join(level, col("level") == col("levelName"), "left").select("*")

In [ ]:
level_joined_df = level_joined_df.drop('levelName')

In [ ]:
website_joined_df = level_joined_df.join(website, col("website") == col("websiteName"), "left").select("*")

In [68]:
from pyspark.sql.functions import lit
course = website_joined_df.withColumn('expiredDate', lit('2099-01-01')) \
                .withColumn('rowStatus', lit(1)) \
                .withColumn('courseKey', monotonically_increasing_id()) \
                .withColumn('courseId',  monotonically_increasing_id())

In [69]:
course = course.drop('level', 'websiteName', 'website')

In [75]:
course = course.select(col('courseKey'), col('courseId'), col('courseName'), col('enroll') ,col('rating'), col('fee'),col('time'), col('link'), col('levelId'), col('websiteId'), col('createDate'), col('expiredDate'), col('rowStatus'))

In [78]:
course.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_course')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [ ]:
instructor = instructor.withColumn('courseKey', monotonically_increasing_id())
instructor = instructor.select(col('courseKey'), explode("instructor").alias("instructor"))

In [97]:
instructor_joined_df = instructor.join(instructorNode, col("instructor") == col("instructorName"), "left").select("*")

In [102]:
instructor_joined_df = instructor_joined_df.select('courseKey', 'instructorId')

In [104]:
instructor_joined_df.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','bridge_instructor_course')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [142]:
df3 = df2.withColumn('id', monotonically_increasing_id())
subtitle = df3.select(col('id').alias('courseKey'), explode(col("Subtitle")).alias("language"))

In [143]:
subtitle = subtitle.select(col('courseKey'), lower(col('language')).alias('language'))
subtitle = subtitle.withColumn('language', lower(col('language'))) \
            .withColumn("language", f.regexp_replace(f.col("language"), "subtitles: ", "").alias("language")) \
            .withColumn("language", f.regexp_replace(f.col("language"), "subtitles:", "").alias("language")) \
            .withColumn("language", f.regexp_replace(f.col("language"), "none", "english").alias("language"))

In [144]:
subtitle_joined_df = subtitle.join(subtitleNode, col("language") == col("languageName"), "left").select("*")

In [146]:
subtitle_joined_df = subtitle_joined_df.select('languageId', 'courseKey')

In [147]:
subtitle_joined_df.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','bridge_language_course')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [149]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
#from transformers import BertForTokenClassification

In [150]:
from sparknlp.annotator import *
MODEL_NAME = '/home/worker2/Desktop/thesis/NER/NERModel_config'
bert = BertForTokenClassification.loadSavedModel(
     '{}/converting/saved_model/1'.format(MODEL_NAME),
     spark
 )\
 .setInputCols(["document",'token'])\
 .setOutputCol("ner")\
 .setCaseSensitive(True)\
 .setMaxSentenceLength(128)

2023-02-26 20:11:32.961746: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:32] Reading SavedModel from: /home/worker2/Desktop/thesis/NER/NERModel_config/converting/saved_model/1
2023-02-26 20:11:33.028150: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:55] Reading meta graph with tags { serve }
2023-02-26 20:11:33.028226: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:93] Reading SavedModel debug info (if present) from: /home/worker2/Desktop/thesis/NER/NERModel_config/converting/saved_model/1
2023-02-26 20:11:33.028417: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 20:11:33.262610: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:2

In [151]:
bert.write().overwrite().save("./{}".format(MODEL_NAME))

In [152]:
tokenClassifier_loaded = BertForTokenClassification.load("./{}".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

In [ ]:
tokenClassifier_loaded.getClasses()

In [170]:
from sparknlp.base import *
document_assembler = DocumentAssembler() \
    .setInputCol('description') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

converter = NerConverter()\
    .setInputCols(["document","token","ner"])\
    .setOutputCol("ner_span")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    tokenClassifier_loaded,
    converter
])

In [160]:
import ftfy
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
def ftfy_udf(text):
    return ftfy.fix_text(text)

fix_text = udf(ftfy_udf, StringType())

In [165]:
from pyspark.sql.functions import lower, col
from pyspark.sql.functions import trim,ltrim,rtrim
import pyspark.sql.functions as f
df4 = df3.select(df.SkillWillLearn.alias('description'))
df4 = df4.withColumn('description', lower(col('description'))) \
        .withColumn('description', f.regexp_replace(col("description"), "[\n\r\b\f\t]", "")) \
        .withColumn('description', f.regexp_replace(col("description"), "®", "")) \
        .withColumn('description', f.regexp_replace(col("description"), "'s", "")) \
        .withColumn('description', f.regexp_replace(col("description"), ";", " ")) \
        .withColumn('description', fix_text(col('description'))) \
        .withColumn('description', f.decode('description','UTF-8'))

In [171]:
result = pipeline.fit(df4).transform(df4)

In [172]:
from pyspark.sql.functions import split, col, monotonically_increasing_id
result = result.withColumn("id", monotonically_increasing_id())

In [173]:
import pyspark.sql.functions as F
result_last = result.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities"), 'id') \
.select(F.expr("entities['result']").alias("name"), 
        F.expr("entities['metadata'].entity").alias("entity"), 'id')

In [210]:
result_last = result_last.orderBy("id").where(result_last.entity != 'EP]')

In [211]:
competency = result_last.select(col('name').alias('competencyName'), col('entity').alias('type'))

In [212]:
competency = competency.dropDuplicates(['competencyName', 'type']).select('competencyName','type')
competency = competency.withColumn('competencyId',monotonically_increasing_id())
competency = competency.select('competencyId', 'competencyName', 'type')

In [181]:
competency.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_competency')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [ ]:
fact_course = result_last.dropDuplicates(['name','entity','id']).select(col('id').alias('courseKey'), col('name').alias('competency'), col('entity')).orderBy('courseKey')

In [ ]:
fact_course = fact_course.join(competency, (col('competency') == col('competencyName')) & (col('entity') == col('type')), 'left').select('*')

In [ ]:
fact_course = fact_course.select('courseKey', 'competencyId')
fact_course = fact_course.withColumn('is_require', lit(0))

In [188]:
fact_course.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','fact_course')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [189]:
from sparknlp.base import *
document_assembler = DocumentAssembler() \
    .setInputCol('description') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

converter = NerConverter()\
    .setInputCols(["document","token","ner"])\
    .setOutputCol("ner_span")

pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    tokenClassifier_loaded,
    converter
])

In [191]:
df4 = df3.select(df3.skillrequirement.alias('description'))
df4 = df4.withColumn('description', lower(col('description'))) \
        .withColumn('description', f.regexp_replace(col("description"), "[\n\r\b\f\t]", "")) \
        .withColumn('description', f.regexp_replace(col("description"), "®", "")) \
        .withColumn('description', f.regexp_replace(col("description"), "'s", "")) \
        .withColumn('description', f.regexp_replace(col("description"), ";", " ")) \
        .withColumn('description', fix_text(col('description'))) \
        .withColumn('description', f.decode('description','UTF-8'))

In [198]:
require_result = pipeline.fit(df4).transform(df4)

In [199]:
require_result = require_result.withColumn("id", monotonically_increasing_id())
require_last = require_result.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities"), 'id') \
.select(F.expr("entities['result']").alias("name"), 
        F.expr("entities['metadata'].entity").alias("entity"), 'id')

In [200]:
require_last = require_last.orderBy("id").where(require_last.entity != 'EP]')

In [206]:
competency_require = require_last.select(col('name').alias('competencyName'), col('entity').alias('type'))

In [207]:
competency_require = competency_require.dropDuplicates(['competencyName', 'type']).select('competencyName','type')
competency_require = competency_require.withColumn('competencyId',monotonically_increasing_id())
competency_require = competency_require.select('competencyId', 'competencyName', 'type')

In [221]:
from pyspark.sql.functions import max
max_key = competency.select(max('competencyId').alias('maxkey')).collect()[0][0]
max_key

613

In [227]:
set_skill_learn = competency.select('competencyName', 'type')
set_skill_require = competency_require.select('competencyName', 'type')

In [229]:
result_df = set_skill_require.exceptAll(set_skill_learn)

In [ ]:
result_df = result_df.withColumn('competencyId', max_key + 1 + monotonically_increasing_id())
result_df.show()

In [234]:
result_df = result_df.select('competencyId','competencyName', 'type')

In [235]:
result_df.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','dim_competency')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()

In [236]:
fact_course_require = require_last.dropDuplicates(['name','entity','id']).select(col('id').alias('courseKey'), col('name').alias('competency'), col('entity')).orderBy('courseKey')

In [238]:
fact_course_require = fact_course_require.join(competency_require, (col('competency') == col('competencyName')) & (col('entity') == col('type')), 'left').select('*')

In [240]:
fact_course_require = fact_course_require.select('courseKey', 'competencyId')
fact_course_require = fact_course_require.withColumn('is_require', lit(1))

In [242]:
fact_course_require.write.format('jdbc')\
    .mode("Append")\
    .option("url", CONNECTION_STRING)\
    .option('driver',CONNECTOR_TYPE) \
    .option('dbtable','fact_course')\
    .option("user", SQL_USERNAME)\
    .option('password', SQL_PASSWORD)\
    .save()